# Bayesian Spam Filter
### -- by Chengrui Charlie Zheng
## Introduction
A spam filter is a filter in your email to filter spams, the mails you do not want to receive, out of hams, the legitimate emails you want. There are multiple ways of implementing a spam filter. I will use Naive Bayes Classifier and Bag-of-Words model to implement a Bayesian spam filter, the oldest kind of spam filter. This passage will walk you through the process of implementation, training and testing.
## Email and Text Cleaning
When a email comes into your mailbox, there are a lot of features to determine whether the email is a spam or not. For a simple demonstration, I will only be analyzing the words in the body of a email, by calculating the probability of the word occuring in a spam. Here, I will first create a class of email as in the following code. The class consists of a constructor, which cleans the text, getters and setters for the attributes.

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

#This class simulates the emails
class email():

    def __init__(self, id, fileName, score=0):
        #this is the id of the email
        self.id = id
        #read the file
        f = open(fileName,'rt')
        #make all words lowercase
        text = f.read()
        #split the text into words and strip all punctuation
        words = word_tokenize(text)
        #strip stop words and number, only remain the stems
        en_stopwords = stopwords.words('english')
        porter = PorterStemmer();
        self.words = [porter.stem(w.lower()) for w in words if w.isalpha() and not w in en_stopwords]
        #this is the score of its spamicity
        self.score = score
        
    def getId(self):
        return self.id

    def getWords(self):
        return self.words

    def getScore(self):
        return self.score

    def setScore(self, newScore):
        self.score = newScore

Here is a detailed explanation of the constructor per se. An email should have an "ID", a string list of "words", and a "score" indicating its spam score. The higher the spam score is, the more likely the mail is spam. When a new mail comes in, we will first set its score to be 0 by default. As for "words", we will do the  text cleaning by using 'nltk' module in python. First, it will read the text from the file, and tokenize the text to split the text into words. For example, if we have a sentence "What do you mean by tokenizing?". Aftering tokenizing the text, it will extract every tokens(a word, a number or a punctuation), and make the tokens into a string list.

In [3]:
text = "What do you mean by tokenizing the text?"
tokenized = word_tokenize(text)
tokenized

['What', 'do', 'you', 'mean', 'by', 'tokenizing', 'the', 'text', '?']

Next, we will filter the stop words out of the tokenized string list. Stop words are the commonly used but semantically insignificant words such as "a" and "by". Here are the new string list from our last example without stop words.

In [4]:
for w in tokenized:
    if w in stopwords.words('english'):
        tokenized.remove(w)
tokenized

['What', 'you', 'mean', 'tokenizing', 'text', '?']

Also, we only want to keep the stem of each words by making the letters lowercase, removing the inflection and some derivation; so that "tokenized", "tokenizing" and "tokenize" will be counted as the same word "token".

In [5]:
stems = [PorterStemmer().stem(w) for w in tokenized]
stems

['what', 'you', 'mean', 'token', 'text', '?']

Ultimately, we will remove the numbers and punctuations in the string list. After the text is clean, we can pass the emails to the filter.

In [6]:
[w for w in stems if w.isalpha()]

['what', 'you', 'mean', 'token', 'text']

## Filter and Naive Bayes Classifier
This filter will be using a Naive Bayes Classifier to classify the comming emails. The more detailed application of  Bayes' Theorem will be explained below. Here is the class of filter.

In [7]:
class filter():

    def __init__(self, threshold):
        # this is the threshold of the spam score
        self.threshold = threshold
        # the inbox stores all hams
        self.inbox = []
        # spams store all spams
        self.spams = []
        # the bag of words of spams
        self.spamBOW = {}
        # the bag of words of hams
        self.hamBOW = {}
    
    #this function determines the spamicity of each word
    def wordSpamicity(self, word):
        #the frequency of the word appeared in spams
        wordInSpam = 0
        #the frequency of the word appeared in hams
        wordInHam = 0
        #checking the bags of words to set the frequency
        if word in self.spamBOW.keys(): wordInSpam = self.spamBOW[word]
        if word in self.hamBOW.keys(): wordInHam = self.hamBOW[word]
        #calculating the probability of a spam word
        pws = float(wordInSpam/len(self.spams))
        pwh = float(wordInHam/len(self.inbox))
        if (pws + pwh) == 0: psw =0
        else: psw = float(pws/(pws+pwh))
        return psw
        
    #this function calcuates the spam score of the email according to its words' spamicity
    def spamScore(self,mail):
        score = 0;
        #getting the spamicity of each word
        for w in mail.getWords():
            spamicity = self.wordSpamicity(w)
            if spamicity >= 0.9: score += 3
            if spamicity >= 0.75 and spamicity < 0.9: score += 2
            if spamicity >= 0.5 and spamicity < 0.75: score += 1
        return score
    
    #this function receives a new email, calculates its spam score, update the bags of words, and sort the email
    def receive(self, mail):
        score = self.spamScore(mail)
        mail.setScore(score)
        if score > self.threshold: 
            self.spams.append(mail)
            print(mail.getId() + ' is a spam. Its Spam score is ' + str(mail.getScore()))
            self.addWords(self.spamBOW, mail.getWords())
        else: 
            self.inbox.append(mail)
            print(mail.getId() + ' is not a spam. Its Spam score is ' + str(mail.getScore()))
            self.addWords(self.hamBOW, mail.getWords())
    
    #this function trains the filter
    def train(self, mail, isSpam):
        if isSpam is True: 
            self.spams.append(mail)
            self.addWords(self.spamBOW, mail.getWords())
        else: 
            self.inbox.append(mail)
            self.addWords(self.hamBOW, mail.getWords())
    
    @staticmethod
    def addWords(bow, words):
        checked = set()
        for w in words:
            if w not in checked: 
                checked.add(w)
                if w not in bow.keys(): bow.update({w:1})
                else: 
                    recur = bow[w] + 1
                    bow.update({w:recur})

### Constructor
In the constructor, we can set a filter's threshold of the spam score. Hams with scores lower than the threshold will be passed into "inbox", an email list resembling the inbox in your mailbox. Spams with scores higher than the threshold will be passed into "spams", an email list of spams. There are also 2 bag-of-words, "spamBOW" for spams and "hamBOW" for hams. The bag-of-words model is a hashmap or a dictionary containing the words appeared in mails as keys, and the number of mails each word appeared as values. The example of the bag-of-words will be illustrated in the Training section.

### Naive Bayes Foundation
After the filter is constructed, it will calculate the spamicity, the probability that a message containing a given word is spam in "wordSpamicity" function. The calculation process is based on Bayes' Theorem:
$$\Pr(S|W)=\frac{\Pr(W|S)\Pr(S)}{\Pr(W)}$$
$Pr(S|W)$ is the probability that the email is a spam, knowing that the target word is in it. Namely, the probability that the email containing the target word is a spam.<br>
$Pr(W|S)$ is the probability that the target word is in the email, knowing that the email is a spam. Namely, the probability that the target word appears in a spam.<br>
$Pr(S)$ is the probability that the given email is a spam.<br>
$Pr(W)$ is the probability that the word appears in any emails.<br>

To futher break down $Pr(W)$, we can calculate it as:
$$\Pr(W)={\Pr(W|S)\Pr(S)}+{\Pr(W|H)\Pr(H)}$$
Because the word appears either appears in a ham or spam, we can calculate $Pr(w)$ as the sum of the probability that the word appears in hams, and the probability that the word appears in spams.

The probability that the word appears in spams can be calculated as $Pr(W|S)Pr(S)$. Namely, the probability that the probability that the target word appears in a spam multiplied by the probability that the given email is a spam.<br>

The probability that the word appears in hams can be calculated as $Pr(W|H)Pr(H)$. Namely, the probability that the probability that the target word appears in a ham multiplied by the probability that the given email is a ham.

To plug in the formula of $Pr(w)$ into the formula of Bayes' Theorem, we can the formula:
$$\Pr(S|W)=\frac{\Pr(W|S)\Pr(S)}{\Pr(W|S)\Pr(S)+\Pr(W|H)\Pr(H)}$$

Now, the question is what are $Pr(S)$ and $Pr(H)$? If we are taking more features into account, such as probability of the email from a certain email address is 80% a spam and 20% a ham, we can take the exact value into calculation. However, because we are only using Naive Bayes Classifier to analyze the body of emails, we have unbiased hypothesis on whether a email is spam or ham. Therefore, $\Pr(S)=\Pr(H)=0.5$.
Since $Pr(S) = Pr(H)$, we can simplify the formula above as:
$$\Pr(S|W)=\frac{\Pr(W|S)}{\Pr(W|S)+\Pr(W|H)}$$
### Spamicity and Spam Score
Now we found a way to calculate the probability that the email containing the target word is a spam. In our context, $Pr(W|S)$ can be calculated as the number of spams containing the target word divided by the total number of spams. We can look up the word in "spamBOW" to obtain the number of spams containing the target word, and get the length of "spams" to obtain the total number of spams. 

On the other hand, $Pr(W|H)$ can be calculated as the number of hams containing the target word divided by the total number of hams. We can look up the word in "hamBOW" to obtain the number of hams containing the target word, and get the length of "inbox" to obtain the total number of hams. 

We also need to consider an edge case. What if the filter has never seen the target word before? Since the target word is not in both of the bag-of-words. $Pr(W|S)$ and $Pr(W|H)$ will be 0, resulting in the divisor of our formula, $Pr(W|S) + Pr(W|H)$ to be 0. To solve this problem, we will be make the filter be more lenient and let the word pass the filter with a spamicity of 0.

After figuring out the way to calculate the spamicity of each word, the "spamScore" function will loop through each word of the email and calculate the spam score of email according to the spamicity of each word. 
* If a word's spamicity is from 0 to 0.5, then the word is safe and will not add any points to the spam score of the email. 
* If a word's spamicity is from 0.5 to 0.75, it will add 1 points to the spam score of the email. 
* If a word's spamicity is from 0.75 to 0.9, it will add 2 points to the spam score of the email. 
* If a word's spamicity is from 0.9 to 1, it will add 3 points to the spam score of the email.

The last step is to receive a email. In the "receive" function, the filter will receive the email, calculate the spam score of that email and sort it to either "inbox" as a ham, or "spams" as a spam. This function will also further train the filter by adding the words of the email to the corresponding bag-of-words. Note: a certain word may appear multiple times in a email, but we are only adding it once a email, because the value of that word in the bag-of-words denotes the number of emails the word appears.
## Training
After constructing the 2 classes, we need to train the filter to let it work. For a real filter, we need a great amount of spams and hams to train it. But for demonstration purpose, I will only use 2 spams and 2 hams for training. The training spams will be subscription confirmation emails we hate to receive. Here is the text of one of the training spams.

*Thank you for subscribing. To begin receiving the newsletter, you must first confirm your subscription.*

Now, we will pass the training data in a filter with a threshold of 10.

In [8]:
#creating training data
spam1 = email('spam1_train','spam1.txt')
spam2 = email('spam2_train','spam2.txt')
ham1  = email('ham1_train','ham1.txt')
ham2  = email('ham2_train','ham2.txt')
#creating the filter and set the threshold to 10
spamFilter = filter(10)
#training
spamFilter.train(spam1, True)
spamFilter.train(spam2, True)
spamFilter.train(ham1, False)
spamFilter.train(ham2, False)

Now the filter is trained, let us explore what are the bag-of-words of spams look like now.

In [9]:
spamFilter.spamBOW

{'thank': 1,
 'subscrib': 1,
 'to': 2,
 'begin': 1,
 'receiv': 1,
 'newslett': 1,
 'must': 1,
 'first': 1,
 'confirm': 1,
 'subscript': 2,
 'you': 1,
 'purchas': 1,
 'follow': 1,
 'free': 1,
 'trial': 1,
 'charg': 1,
 'after': 1,
 'end': 1,
 'renew': 1,
 'unless': 1,
 'cancel': 1,
 'nov': 1,
 'learn': 1,
 'review': 1}

We can see that the 2 training spams are added to the "spams" of "spamFilter"

In [10]:
print('spams:')
[mail.getId() for mail in spamFilter.spams]

spams:


['spam1_train', 'spam2_train']

The 2 training hams are also added to the "inbox" of "spamFilter"

In [11]:
print('hams:')
[mail.getId() for mail in spamFilter.inbox]

hams:


['ham1_train', 'ham2_train']

We can calculate the spamicity of a word's stem. For example, let us try "subscript" and "thank".

In [12]:
print('the spamicity of "subscript" is ' + str(spamFilter.wordSpamicity('subscript')))
print('the spamicity of "thank" is ' + str(spamFilter.wordSpamicity('thank')))

the spamicity of "subscript" is 1.0
the spamicity of "thank" is 0.5


"Subscript" gets the spamicity of 1 because it appears twice in the 2 spams but not in any hams. "Thank" gets the spamicity of 0.5 because it appears once in the 2 spams and once in the 2 hams.
We can see the spam scores of these 4 training emails.

In [13]:
print('the score of spam1_train is ' + str(spamFilter.spamScore(spam1)))
print('the score of spam2_train is ' + str(spamFilter.spamScore(spam2)))
print('the score of ham1_train is ' + str(spamFilter.spamScore(ham1)))
print('the score of ham2_train is ' + str(spamFilter.spamScore(ham2)))

the score of spam1_train is 24
the score of spam2_train is 70
the score of ham1_train is 4
the score of ham2_train is 1


## Testing

Let us pass the testing data in the filter to check if the filter is well-trained.
This is the text of the testing spam, another subscription confirmation email:

*Please Confirm Subscription. Yes, subscribe me to this list. If you received this email by mistake, simply delete it. You won't be subscribed if you don't click the confirmation link above. For questions about this list, please contact:*

This is the text of the testing ham, a email about COVID-19:

*We understand that the recent news and uncertainty surrounding the COVID-19 situation may have caused you to re-think your travel plans and future travel options. Whether you have a trip booked or are planning upcoming travel, we will do whatever we can to support you. We are continually monitoring the situation, including travel restrictions and updates to travel policies that may impact you.*

In [14]:
#creating testing data
spam3 = email('spam3_test','spam3.txt')
ham3  = email('ham3_test','ham3.txt')
#testing
spamFilter.receive(spam3)
spamFilter.receive(ham3)

spam3_test is a spam. Its Spam score is 15
ham3_test is not a spam. Its Spam score is 0


The filter successfully filtered "spam3_test" as a spam and passed "ham3_test" as a ham.

However, this filter is only for demonstration purpose of the application of simple text cleaning and Naive Bayes Classifier. In the real life, we need to consider more features other than contents, such as senders' addresses and subject. We need more complicated models, like decision trees, to build a spam filter like that in Gmail.